In [1]:
import spacy
import timeit
import math
import pandas as pd
import matplotlib.pyplot as plt
from os import path
from collections import Counter
from lxml import etree
from glob import glob
from unicodedata import normalize

In [2]:
%matplotlib inline

In [3]:
nlp = spacy.load('fr_core_news_lg')

In [4]:
def est_canon(tree):
    if tree.find(".//profileDesc") is not None:
        profil = tree.find(".//profileDesc")
        if profil.get("tag") == "canon":
            return True
        else:
            return False

In [5]:
def lemmatize_rolling(path):
    list_lemma = []
    with open(path, encoding="utf8") as file:
        tree = etree.parse(file)
        tag = est_canon(tree)
        if tag == True:
            print("canon")
        else:
            print("non_canon")
        if tree.findall(".//p"):
            for paragraphe in tree.findall(".//p"):
                if paragraphe.text:
                    clean_text = normalize("NFKD", paragraphe.text)
                    docs = nlp(clean_text)
                    for token in docs:
                        if token.pos_ != "PUNCT" and "SPACE" and "X" and "SYM":
                            list_lemma.append(token.lemma_)
    return list_lemma, tag

In [9]:
#def shannon(rolling_lists_bigrams):
 #   shannon_measures = []
  #  for list_bigrams in rolling_lists_bigrams:
   #     shannon_measures.append(shannon_diversity(list_bigrams))
    #return shannon_measures

In [10]:
path_test = 'corpus_test/1829_Hugo-Victor_Le-dernier-jour-d-un-condamne.xml'

In [37]:
def rollingnwords(list_lemma, n):
    i = 0 # i stocke l'indice auquel on est dans le rolling
    list_rolling = []
    while i-n < len(list_lemma):
        list_rolling.append(list_lemma[i:i+n])
        i+=n
    return list_rolling    

In [64]:
def shannon_diversity(rolling_list_bigram):
    shannon_measures = []
    # initialisation de l'indice de shannon
    # on recupere la proportion pi de chaque bigram par rapport à tous les autres bigrams
    for list_bigram in rolling_list_bigram:
        shannon_sum = 0
        dict_conteur = Counter(list_bigram)
        for bigram in list_bigram:
            prop = dict_conteur[bigram]/len(list_bigram)
            # on met à jour l'indice de shannon
            shannon_courant = prop * (math.log(prop, 2))
            shannon_sum += shannon_courant
        shannon_measures.append(shannon_sum * -1)
    return shannon_measures

In [70]:
def bigrammize_rolling(rolling_list_lemma):
    rolling_list_bigram = []
    list_bigram = []
    for list_lemma in rolling_list_lemma:
        for indice_lemma in range(len(list_lemma)-1):
            bigram = list_lemma[indice_lemma]+'_'+list_lemma[indice_lemma+1]
            list_bigram.append(bigram)
        print(list_bigram)
        rolling_list_bigram.append(list_bigram)
    return rolling_list_bigram

In [71]:
window = 1000
list_lemma_hugo, canon = lemmatize_rolling(path_test)
rolling_list_lemma = rollingnwords(list_lemma_hugo, window)
rolling_list_bigram = bigrammize_rolling(rolling_list_lemma)
results = shannon_diversity(rolling_list_bigram)

canon


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




['il_n’', 'n’_y', 'y_avoir', 'avoir_en', 'en_tête', 'tête_de', 'de_première', 'première_édition', 'édition_de', 'de_ce', 'ce_ouvrage', 'ouvrage_publié', 'publié_d’', 'd’_abord', 'abord_sans', 'sans_nom', 'nom_d’', 'd’_auteur', 'auteur_que', 'que_le', 'le_quelque', 'quelque_ligne', 'ligne_qu’', 'qu’_on', 'on_aller', 'aller_lire', 'lire_il', 'il_y', 'y_avoir', 'avoir_deux', 'deux_manière', 'manière_de', 'de_se', 'se_rendre', 'rendre_compte', 'compte_de', 'de_l’', 'l’_existence', 'existence_de', 'de_ce', 'ce_livre', 'livre_ou', 'ou_il', 'il_y', 'y_avoir', 'avoir_avoir', 'avoir_en', 'en_effet', 'effet_un', 'un_liasse', 'liasse_de', 'de_papier', 'papier_jaune', 'jaune_et', 'et_inégal', 'inégal_sur', 'sur_lequel', 'lequel_on', 'on_avoir', 'avoir_trouvé', 'trouvé_enregistrées', 'enregistrées_un', 'un_à', 'à_un', 'un_le', 'le_dernière', 'dernière_pensée', 'pensée_d’', 'd’_un', 'un_misérable', 'misérable_ou', 'ou_il', 'il_s’', 's’_être', 'être_rencontré', 'rencontré_u

il_n’
il_n’
il_n’
il_n’
il_n’
il_n’
il_n’
il_n’
il_n’
il_n’
il_n’
il_n’
il_n’
il_n’
il_n’
il_n’
il_n’
il_n’
il_n’
il_n’
il_n’
il_n’
il_n’
il_n’
il_n’
il_n’
il_n’
il_n’
il_n’
il_n’
il_n’
il_n’
il_n’
il_n’
il_n’


In [69]:
results

[119.11771356373757,
 119.11771356373757,
 119.11771356373757,
 119.11771356373757,
 119.11771356373757,
 119.11771356373757,
 119.11771356373757,
 119.11771356373757,
 119.11771356373757,
 119.11771356373757,
 119.11771356373757,
 119.11771356373757,
 119.11771356373757,
 119.11771356373757,
 119.11771356373757,
 119.11771356373757,
 119.11771356373757,
 119.11771356373757,
 119.11771356373757,
 119.11771356373757,
 119.11771356373757,
 119.11771356373757,
 119.11771356373757,
 119.11771356373757,
 119.11771356373757,
 119.11771356373757,
 119.11771356373757,
 119.11771356373757,
 119.11771356373757,
 119.11771356373757,
 119.11771356373757,
 119.11771356373757,
 119.11771356373757,
 119.11771356373757,
 119.11771356373757,
 119.11771356373757,
 119.11771356373757,
 119.11771356373757]